In [8]:
import os
import sys
import urllib, io
import pickle
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
proj_dir = os.path.abspath('../..')
import importlib

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy import stats
from scipy.spatial import distance
from scipy import ndimage
from scipy.stats import entropy
from random import random
import itertools

from collections import Counter
import json
import re
import ast

from io import BytesIO
import base64

from PIL import Image, ImageOps, ImageDraw, ImageFont 
import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from matplotlib import colors

import plotly
import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

from collections import Counter

# import drawing_utils as drawing 
# import block_construction_helpers as bc 

impor

In [6]:
# setup

structures = {
    'wa':
    'wb':
    'wc':
    'wd':
    'ta':
    'tb':
    'tc':
    'td':
}



ambiguous_stims = {'wawb-tatb'}



conditions = {'A': [],
              'B': []}





In [7]:
# generate synthetic data

df = pd.DataFrame()

for ppt in range(0,2):
    
    for trial in range(1,12):
        
        
    

    


""
